In [8]:
import json
from joblib import load
from pathlib import Path

from config import Config
from tools.tester import calc_eer
from tools.dataset_loader import get_data, get_stat_features

## Load precomputed stat. features and labels

In [9]:
assert Path('data/train').exists() and\
       Path('data/dev').exists() and\
       Path('data/val').exists(), 'Download precomputed features'

with open('configs/included_features.json', 'r') as config:
    included_features = json.load(config)
    
save_path = Path('data')
*_, val_x = get_stat_features(path=save_path, included_features=included_features)
*_, (_, val_y) = get_data(dataset_path=Config.dataset_path, protocol_paths=Config.protocols_paths)

  0%|          | 0/3 [00:00<?, ?it/s]

Train shape: (25380, 780)
Dev shape: (24844, 780)
Eval shape: (71237, 780)
Loading ASVspoof 2019 LA testing protocols...


100%|██████████| 3/3 [00:00<00:00,  7.03it/s]


## Load pretrained models

In [10]:
assert Path('pretrained_models/random_forest_train_dev').exists() and\
       Path('pretrained_models/cat_boost_train_dev').exists() and\
       Path('pretrained_models/light_gbm_train_dev').exists(), 'Download pretrained models'

rf = load('pretrained_models/random_forest_train_dev')
cb = load('pretrained_models/cat_boost_train_dev')
lgbm = load('pretrained_models/light_gbm_train_dev')

## Evaluate EER on the validation set

In [11]:
for clf, clf_name in zip([rf, cb, lgbm], ['Random forest', 'CatBoost', 'Light GBM']):
    pred_proba = clf.predict_proba(val_x).T[1]
    eer, threshold = calc_eer(val_y, pred_proba)
    print(f'{clf_name}\nValidation EER: {round(eer, 2)}\nThreshold: {round(threshold, 2)}\n')

Random forest
Validation EER: 9.55
Threshold: 0.37

CatBoost
Validation EER: 4.21
Threshold: 0.34

Light GBM
Validation EER: 3.93
Threshold: 0.34

